In [1]:
import tensorflow as tf
import numpy as np
import time
import os

In [2]:
from keras.datasets import mnist, cifar10
(img_train, label_train), (img_test, label_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
def Conv_block(inputs, filters, scope_num):
    with tf.variable_scope('{}/conv_layer'.format(model, scope_num)):
        conv = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=[3, 3], 
                                padding='SAME', activation=tf.nn.relu, 
                                name='conv{}'.format(scope_num))
        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], 
                                       padding='SAME', strides=2, 
                                       name='pool{}'.format(scope_num))
    return pool

In [4]:
def get_accuracy(sess, logits, img, label, batch_size):    
    if len(img) < batch_size:
        logit = sess.run(logits, feed_dict={X: img, training: False})
    else:
        total_batch = int(len(img) / batch_size) + 1
        num_label = int(logits.shape[-1])
        logit = np.array([], dtype=np.int64).reshape(0, num_label)
        idxs = range(len(img))
        for i in range(total_batch):
            if i < (total_batch - 1):
                idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
            else:
                idxs_i = idxs[i * batch_size : len(img)]
            feed_dict = {X: img[idxs_i], training: False}
            logit = np.concatenate((logit, sess.run(logits, feed_dict=feed_dict)), axis=0)

    correct_pred = np.equal(np.argmax(logit, 1), label).astype(np.float32)
    accuracy = np.mean(correct_pred)
    
    return accuracy, logit, correct_pred

In [5]:
model = 'basic'
img_row, img_col = img_train.shape[1], img_train.shape[2]
img_depth = 1

img_train = img_train.reshape(img_train.shape[0], img_row, img_col, img_depth)
img_test = img_test.reshape(img_test.shape[0], img_row, img_col, img_depth)
img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

In [6]:
X = tf.placeholder(tf.float32, [None, img_row, img_col, img_depth])
Y = tf.placeholder(tf.int64, [None])
training = tf.placeholder(tf.bool)

In [7]:
# build network
# convolution layers

block1 = Conv_block(inputs=X, filters=32, scope_num=1)
block2 = Conv_block(inputs=block1, filters=32, scope_num=2)
conv = Conv_block(inputs=block2, filters=32, scope_num=3)

In [8]:
# fully connected layers

flat_num = int(conv.shape[1] * conv.shape[2] * conv.shape[3])
flat = tf.reshape(conv, [-1, flat_num])

with tf.variable_scope('{}/fc_layer'.format(model)):
    dense1 = tf.layers.dense(inputs=flat, units=512, 
                            activation=tf.nn.relu, name='dense1')
    drop1 = tf.layers.dropout(inputs=dense1, rate=0.25, training=training)
    dense2 = tf.layers.dense(inputs=drop1, units=512, 
                            activation=tf.nn.relu, name='dense2')
    drop2 = tf.layers.dropout(inputs=dense2, rate=0.25, training=training)
    logits = tf.layers.dense(inputs=drop2, units=10, name='logit')        

In [11]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
# hyper prameter
batch_size = 128
epochs = 5
CHECK_POINT_DIR = './save_model'

In [13]:
config=tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allocator_type='BFC'
config.log_device_placement=False
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [14]:
# Restore and Save

saver = tf.train.Saver()
checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)

if checkpoint and checkpoint.model_checkpoint_path:
    try:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    except:
        print("Error on loading old network weights")
else:
    print("Could not find old network weights")

Could not find old network weights


In [16]:
# train

print(time.asctime())
print ('Learning Started.')

total_start = time.time()
max_accuracy = 0.0
for epoch in range(epochs):
    epoch_start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size) + 1
    
    idxs = np.random.permutation(range(len(img_train)))
    for i in range(total_batch):
        if i < (total_batch - 1):        
            idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
        else:
            idxs_i = idxs[i * batch_size : len(img_train)]
            
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i], training: True}    
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print ('=====================================')
    print ('Epoch : {:02d} , Cost : {:.5f}'.format(epoch + 1, avg_cost))
    train_accuracy, _, _ = get_accuracy(sess, logits, img_train, label_train, batch_size)
    print ('Train Accuracy : {:.5f}'.format(train_accuracy))
    test_accuracy, _, _ = get_accuracy(sess, logits, img_test, label_test, batch_size)
    print ('Test Accuracy : {:.5f}'.format(test_accuracy))
    print ('Elapsed time of an epoch: {:.5f}'.format(time.time() - epoch_start))
    if test_accuracy > max_accuracy:
        max_accuracy = test_accuracy
        print ('===========Saving network with the best accuracy===========')
        if not os.path.exists(CHECK_POINT_DIR):
            os.makedirs(CHECK_POINT_DIR)
        saver.save(sess, CHECK_POINT_DIR + "/model", global_step=epoch+1)

print ('=====================================')

print ('Final Test Accuracy : {:.5f}'.format(max_accuracy))
print ('Total elapsed time: {:.5f}'.format(time.time() - total_start))

Tue Jan 23 13:43:00 2018
Learning Started.
Epoch : 01 , Cost : 0.02500
Train Accuracy : 0.99440
Test Accuracy : 0.99020
Elapsed time of an epoch: 7.03070
===========Saving network with the best accuracy===========
Epoch : 02 , Cost : 0.02379
Train Accuracy : 0.99583
Test Accuracy : 0.99060
Elapsed time of an epoch: 7.07181
===========Saving network with the best accuracy===========
Epoch : 03 , Cost : 0.01730
Train Accuracy : 0.99240
Test Accuracy : 0.98880
Elapsed time of an epoch: 7.00362
Epoch : 04 , Cost : 0.01723
Train Accuracy : 0.99680
Test Accuracy : 0.98940
Elapsed time of an epoch: 6.95048
Epoch : 05 , Cost : 0.01566
Train Accuracy : 0.99577
Test Accuracy : 0.99010
Elapsed time of an epoch: 6.99260
Final Test Accuracy : 0.99060
Total elapsed time: 35.86638
